In [ ]:
#install.packages("dynlm")
#install.packages("moments")
#install.packages("broom")
#install.packages("FinTS")
#install.packages('rugarch')
#install.packages("tsbox")
#install.packages("metRology")
#install.packages("expm")
#install.packages("MTS")
#install.packages("rmgarch")
#install.packages("Rcpp")
library(expm)
library(Matrix)
library(ggplot2)
library(quantmod)
library(moments)
library(dynlm)
library(broom)
library(FinTS)
library(lubridate)
library(forecast)
library(readxl)
library(MASS)
library(rugarch)
library(tsbox)
library(MTS)
library(rmgarch)
library(Rcpp)


## Librería __Quantmod__

Para el análisis de temas financieros existe una librería de mucha utilidad llamada __quantmod__. En primer lugar esta librería permite acceder a datos financieros de un modo muy simple, es posible decargar series financieras desde _yahoo_, la _FRED (Federal Reserve Economic Data)_, _google_, etc. Por otro lado también es una librería que permite realizar gráficos altamente estéticos con unas cuantas líneas de código. 


In [ ]:
options("getSymbols.warning4.0"=FALSE)

AMZN <-getSymbols("AMZN", src = "yahoo", auto.assign = FALSE)

AMZN <- na.omit(AMZN)

chartSeries(AMZN,TA='addBBands();
                    addBBands(draw="p");
                    addVo();
                    addMACD()', 
                    subset='2021',
                theme="white")

head(AMZN)


Para fines del ejercicio de esta clase, usaremos el valor de la acción ajustado. Esto nos servirá para calcular el rendimiento diario, o puesto en lenguaje de series temporales podemos decir que usaremos la serie en diferencias logarítmicas. 



In [ ]:
plot(AMZN$AMZN.Adjusted)




Una de las preguntas relevantes al observar la serie en diferencias, es si podríamos afirmar que esta serie cumple con el supuesto de homoscedasticidad. 


In [ ]:
logret <- ts(diff(log(AMZN$AMZN.Adjusted))[-1])

plot(logret)


## Value at risk

El Valor en Riesgo, básicamente es un cálculo que nos permite estimar el monto que una acción o portafolio podría perder dada una probabilidad $(1-\alpha)$


In [ ]:
alpha <- 0.05
VaR <- quantile(logret, alpha)

round(VaR, 4)

qplot(logret , geom = 'histogram') + geom_histogram(fill = 'lightblue' , bins = 30) +
  geom_histogram(aes(logret[logret < quantile(logret , 0.05)]) , fill = 'red' , bins = 30) +
  labs(x = 'Daily Returns')


In [ ]:
normal_dist <- rnorm(100000, mean(logret), sd(logret))
VaR_n <- quantile(normal_dist, 0.05)
ES_n <- mean(normal_dist[normal_dist<VaR])
  
ggplot()+
  geom_density(aes(logret, geom ='density', col = 'returns'))+
  geom_density(aes(normal_dist, col = 'normal'))


In [ ]:
vector_ret <- as.vector(logret)

##Kurtosis
round(kurtosis(vector_ret),2)
##Sesgo
round(skewness(vector_ret),2)


$H_o: K=S=0$



In [ ]:
##Prueba de normalidad 
jarque.test(vector_ret)


## Calcular VaR para una distribución t

Como sabemos la distribución normal es un caso particular de la distribución t de student, correspondiente a un caso en el que los Grados de Libertad (_df_) tienden a infinito. Mediante el método de máxima verosimilitud nos aproximamos a una distribución t de student que describe mejor el comportamiento de los retornos de este activo. 


In [ ]:
t.fit <-fitdistr(vector_ret, "t")
round(t.fit$estimate, 6)

library(metRology)
alpha <- 0.05
set.seed(1289)
rvec <- rt.scaled(length(logret), mean=t.fit$estimate[1],
                  sd = t.fit$estimate[2],
                  df = t.fit$estimate[3])

VaR <- quantile(rvec, alpha)
round(VaR, 6)


In [ ]:
ggplot()+
  geom_density(aes(logret, geom ='density', col = 'returns'))+
  geom_density(aes(normal_dist, col = 'normal'))+
    geom_density(aes(rvec, col = 't_simulated'))


# ARCH 

Hasta ahora las distribuciones utilizadas para medir el Valor en Riesgo de este activo, asumen que no existe correlación serial en los retornos diarios de este activo. Observemos un par de gráficas de la función de autocorrelación para corroborar este hecho. 


In [ ]:
plot(logret)
acf(logret)



La idea de clusterización de volatilidad, asume que periodos de alta volatilidad serán seguidos por una alta volatilidad y viceversa. Por esta razón la función de autocorrelación útil para saber si existen clusters de volatilidad, es utilizando el valor absoluto, ya que lo que importa es saber si la serie está autocorrelacionada en la magnitud de los movimientos. 


In [ ]:
acf(abs(logret))



Otra manera de corroborar esta idea, es volviendo IID nuestra serie de datos y observar que de este modo se pierde la autocorrelación serial, lo que refuerza la idea de que en esta serie existen clusters de volatilidad. 



In [ ]:
logret_random <- sample(as.vector(logret),size =  length(logret), replace = FALSE)

acf(abs(logret_random))

par(mfrow = c(1,2))
plot(logret)
plot(logret_random, type = 'l')


Primer enfoque para comprobar aceptar o rechazar la hopótesis de que necesitamos estimar un ARCH(q)



In [ ]:
logret_mean = dynlm(logret~1)

summary(logret_mean)

ehatsq = ts(resid(logret_mean)^2)

ARCH_m = dynlm(ehatsq~L(ehatsq))

summary(ARCH_m)


acf(ARCH_m$residuals)
acf(abs(ARCH_m$residuals))

ArchTest(logret, lags = 1, demean = TRUE)


### ARCH(1)

$Y_t = \mu+\sqrt{h_t}\varepsilon_t$

$h_t = \omega+\alpha_i\varepsilon^2_{t-i}+u_t$

$\varepsilon$~$N(0,1)$


In [ ]:
library(rugarch)

auto.arima(logret)

model.spec = ugarchspec(variance.model = list(model = 'sGARCH' , garchOrder = c(1, 0)), 
                        mean.model = list(armaOrder = c(0,0)), distribution.model = "std")

arch.fit = ugarchfit(spec = model.spec , data = logret, solver = 'solnp')

arch.fit@fit$matcoef

boot.garch <- ugarchboot(arch.fit,
                         method = "Partial",
                         sampling = "raw",  #bootstrap from fitted varepsilon
                         n.ahead = 1,          #simulation horizon
                         n.bootpred = 100000, #number of simulations 
                         solver = "solnp")

## VaR estimado mediante ARCH
rvec <- boot.garch@fseries
alpha <- 0.05
VaR <- quantile(rvec, alpha)
round(VaR,6)

F_inv = qt(0.05, t.fit$estimate[3])

tiempo= time(AMZN[-1])

ggplot()+
  geom_point(aes(y = logret,x = tiempo),colour = 'lightgrey' , size = 2) + 
  geom_line(aes(y = arch.fit@fit$sigma*(F_inv), x = tiempo), colour = 'red') +
  geom_hline(yintercept = sd(logret)*qnorm(0.05) , colour = 'darkgreen' , size = 1.2) + theme_light() + 
  labs(x = '' , y = 'Daily Returns' , title = 'Value at Risk Comparison')


## GARCH(0,1)

$Y_t = \mu+\sqrt{h_t}\varepsilon_t$

$h_t = \omega+\beta_ih^2_{t-i}+u_t$

$\varepsilon$~$N(0,1)$


In [ ]:
model.spec = ugarchspec(variance.model = list(model = 'sGARCH' , garchOrder = c(0,1)), 
                        mean.model = list(armaOrder = c(0,2)), distribution.model = "std")

fit.garch.n = ugarchfit(spec = model.spec, data = logret, solver = "solnp")
fit.garch.n@fit$matcoef

boot.garch <- ugarchboot(fit.garch.n,
                         method = "Partial",
                         sampling = "raw",  #bootstrap from fitted varepsilon
                         n.ahead = 1,          #simulation horizon
                         n.bootpred = 100000, #number of simulations 
                         solver = "solnp")

rvec <- boot.garch@fseries
alpha <- 0.05
VaR <- quantile(rvec, alpha)
VaR

F_inv = qt(0.05, t.fit$estimate[3])

tiempo= time(AMZN[-1])

ggplot()+
  geom_point(aes(y = logret,x = tiempo),colour = 'lightgrey' , size = 2) + 
  geom_line(aes(y = fit.garch.n@fit$sigma*(F_inv), x = tiempo), colour = 'red') +
  geom_hline(yintercept = sd(logret)*qnorm(0.05) , colour = 'darkgreen' , size = 1.2) + theme_light() + 
  labs(x = '' , y = 'Daily Returns' , title = 'Value at Risk Comparison')


### Selección GARCH(p,q) óptimo

$Y_t = \mu+\sqrt{h_t}\varepsilon_t$

$h_t = \omega+\beta_ih_{t-i}+\alpha_i\varepsilon^2_{t-i}+u_t$

$\varepsilon$~$N(0,1)$


In [ ]:
#Criterios de información 

infocriteria(fit.garch.n)


#Selección del modelo óptimo

source("Lag_Opt_GARCH.R")

Lag_Opt_GARCH(ehatsq,4,4)


In [ ]:
# Estimación de modelo óptimo 

model.spec = ugarchspec(variance.model = list(model = 'sGARCH' , garchOrder = c(1,1)), 
                        mean.model = list(armaOrder = c(0,2)), distribution.model = "std")


model.fit = ugarchfit(spec = model.spec , data = logret, solver = 'solnp')

model.fit@fit$matcoef


boot.garch <- ugarchboot(model.fit,
                         method = "Partial",
                         sampling = "raw",  #bootstrap from fitted varepsilon
                         n.ahead = 1,          #simulation horizon
                         n.bootpred = 100000, #number of simulations 
                         solver = "solnp")

## VaR estimado mediante GARCH óptimo
rvec <- boot.garch@fseries
alpha <- 0.05
VaR <- quantile(rvec, alpha)
VaR

F_inv = qt(0.05, t.fit$estimate[3])

tiempo= time(AMZN[-1])

ggplot()+
  geom_point(aes(y = logret,x = tiempo),colour = 'lightgrey' , size = 2) + 
  geom_line(aes(y = model.fit@fit$sigma*(F_inv), x = tiempo), colour = 'red') +
  geom_hline(yintercept = sd(logret)*qnorm(0.05) , colour = 'darkgreen' , size = 1.2) + theme_light() + 
  labs(x = '' , y = 'Daily Returns' , title = 'Value at Risk Comparison')


### Comparativo entre cálculos 



In [ ]:
ggplot()+
  geom_point(aes(y = logret,x = tiempo),colour = 'lightgrey' , size = 2) + 
  geom_line(aes(y = model.fit@fit$sigma*(F_inv), x = tiempo), colour = 'red') +
  geom_line(aes(y = arch.fit@fit$sigma*(F_inv), x = tiempo), colour = 'blue', alpha = 0.25) +
  geom_line(aes(y = fit.garch.n@fit$sigma*(F_inv), x = tiempo), colour = 'blue4') +
  geom_hline(yintercept = sd(logret)*qnorm(0.05) , colour = 'darkgreen') + theme_light() + 
  labs(x = '' , y = 'Daily Returns' , title = 'Value at Risk Comparison')
